# Tutorial on sequence to sequence learning
Taken from **NLP FROM SCRATCH: TRANSLATION WITH A SEQUENCE TO SEQUENCE NETWORK AND ATTENTION** by *Sean Robertson*, available [here](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html)

## Topic
The tutorial example in on language translation. However, this can be taken as an example for training sequence 2 sequence models implementing the **encoder-decoder** approach.

- **encoder**: a network that learns to map a ``sequence`` on a ``vector``
- **decoder**: a network that learns to map a ``vector`` on a ``sequence``

![](seq2seq.png)

## Data
This example is base on language translation. We get the data from [here](https://www.manythings.org/anki/)

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

In [2]:
map_file = '/Users/flint/Data/translation/ita-eng/ita.txt'
M = pd.read_csv(map_file, sep='\t', header=None)[[0,1]]
M.columns = [['eng', 'ita']]

In [3]:
M.tail()

,eng,ita
353276,I know that adding sentences only in your nati...,So che aggiungere frasi soltanto nella sua lin...
353277,I know that adding sentences only in your nati...,So che aggiungere frasi solamente nella sua li...
353278,I know that adding sentences only in your nati...,So che aggiungere frasi solamente nella sua li...
353279,Doubtless there exists in this world precisely...,Senza dubbio esiste in questo mondo proprio la...
353280,Doubtless there exists in this world precisely...,Senza dubbio esiste in questo mondo proprio la...


### Vectorize text using one-hot encoding

In [4]:
from nltk import word_tokenize

In [5]:
SOS_token = 0
EOS_token = 1


class Dictionary:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {SOS_token: "SOS", SOS_token: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def add_sentence(self, sentence):
        for word in word_tokenize(sentence):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

## Over simplify data to have a simple example
We keep only small sentences starting with specific prefixes (and in the original [tutorial](https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html))

In [6]:
MAX_LEN = 10
prefixes = ("i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re ")

In [7]:
ENG = Dictionary('english')
ITA = Dictionary('italian')
pairs = []

for v in M.itertuples():
    e, i = v[1].lower(), v[2].lower()
    if len(word_tokenize(e)) < MAX_LEN and e.startswith(prefixes):
        ENG.add_sentence(e)
        ITA.add_sentence(i)
        pairs.append((e, i))
    else:
        pass

In [8]:
len(pairs)

5129

In [10]:
pairs[100]

('i am a cook.', 'sono una cuoca.')

## Encoder and decoder

In [11]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Encoder
The encoder is a RNN (GRU or LSTM) that outputs some value for every word from the input sentence. For every input word the encoder outputs a vector and a hidden state, and uses the hidden state for the next input word.
![](encoder-network.png)

In [24]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### Decoder
In the simplest seq2seq decoder we use only last output of the encoder. This last output is sometimes called the context vector as it encodes context from the entire sequence. This context vector is used as the initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and hidden state. The initial input token is the start-of-string `<SOS>` token, and the first hidden state is the context vector (the encoder's last hidden state).

![](decoder-network.png)

In [12]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

## Training

In [13]:
def indexes_from_sentence(lang, sentence):
    return [lang.word2index[word] for word in word_tokenize(sentence)]

def tensor_from_sentence(lang, sentence):
    indexes = indexes_from_sentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(
        indexes, dtype=torch.long, device=device).view(-1, 1)

def tensors_from_pair(pair, input_lang, output_lang):
    input_tensor = tensor_from_sentence(input_lang, pair[0])
    target_tensor = tensor_from_sentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [15]:
print(pairs[0][0])
print(word_tokenize(pairs[0][0]))
print([ENG.word2index[x] for x in word_tokenize(pairs[0][0])])
indexes_from_sentence(ENG, pairs[0][0])

i am tom.
['i', 'am', 'tom', '.']
[2, 3, 4, 5]


[2, 3, 4, 5]

In [16]:

tensor_from_sentence(ENG, pairs[0][0])

tensor([[2],
        [3],
        [4],
        [5],
        [1]])

In [18]:
print(pairs[0])
tensors_from_pair(pairs[0], ENG, ITA)

('i am tom.', 'sono tom.')


(tensor([[2],
         [3],
         [4],
         [5],
         [1]]),
 tensor([[2],
         [3],
         [4],
         [1]]))

### Training procedure

To train we run the input sentence through the encoder, and keep track of every output and the latest hidden state. Then the decoder is given the `<SOS>` token as its first input, and the last hidden state of the encoder as its first hidden state.

*Teacher forcing* is the concept of using the real target outputs as each next input, instead of using the decoder’s guess as the next input. Using teacher forcing causes it to converge faster but when the trained network is exploited, it may exhibit instability.

You can observe outputs of teacher-forced networks that read with coherent grammar but wander far from the correct translation - intuitively it has learned to represent the output grammar and can “pick up” the meaning once the teacher tells it the first few words, but it has not properly learned how to create the sentence from the translation in the first place.

Because of the freedom PyTorch’s autograd gives us, we can randomly choose to use teacher forcing or not with a simple if statement. Turn teacher_forcing_ratio up to use more of it.

In [19]:
import random

In [20]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, 
          encoder, decoder, 
          encoder_optimizer, decoder_optimizer, 
          criterion, max_length=MAX_LEN):
    
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(
        max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

### Training procedure

In [21]:
import time
import random

In [22]:
def train_iters(encoder, decoder, input_lang, output_lang,
                n_iters, print_every=1000, 
                plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(
        encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(
        decoder.parameters(), lr=learning_rate)
    training_pairs = [tensors_from_pair(
        random.choice(pairs), input_lang, output_lang)
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()
    
    run = tqdm(list(range(1, n_iters + 1)))

    for iter in run:
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, 
                     encoder, decoder, 
                     encoder_optimizer, decoder_optimizer, 
                     criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
    return plot_losses

In [25]:
hidden_size = 256
encoder1 = EncoderRNN(ENG.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, ITA.n_words).to(device)

plot_losses = train_iters(encoder1, decoder1, ENG, ITA, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

## Usage

In [26]:
def evaluate(encoder, decoder, input_lang, output_lang, 
             sentence, max_length=MAX_LEN):
    with torch.no_grad():
        input_tensor = tensor_from_sentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(
            max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor(
            [[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(
                    output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()

        return decoded_words

In [27]:
for i in np.random.choice(range(len(pairs)), size=(10), replace=False):
    eng, ita = pairs[i]
    print(eng)
    print('correct:\t', ita)
    output_words = evaluate(encoder1, decoder1, ENG, ITA, eng)
    output_sentence = ' '.join(output_words)
    print('predicted:\t', output_sentence)
    print()

i am very interested in french.
correct:	 io sono molto interessato al francese.
predicted:	 sono sono una . <EOS>

she is already married.
correct:	 è già sposata.
predicted:	 non è è . . <EOS>

you are very brave.
correct:	 è molto coraggiosa.
predicted:	 non sono sono . <EOS>

i am a teacher, too.
correct:	 anche io sono un professore.
predicted:	 sono sono una . <EOS>

he is old enough to travel alone.
correct:	 lui è abbastanza grande per viaggiare da solo.
predicted:	 sono sono una . <EOS>

they are content with things as they are.
correct:	 loro sono contente delle cose così come sono.
predicted:	 non sono sono . <EOS>

you are the backing group.
correct:	 siete il gruppo di spalla.
predicted:	 sono sono una . <EOS>

he is good for nothing.
correct:	 lui è buono a nulla.
predicted:	 non è è . . <EOS>

she is someone you can trust.
correct:	 lei è una persona di cui ci si può fidare.
predicted:	 non sono è . <EOS>

you aren't very smart.
correct:	 non siete molto intelligenti.
pr